In [1]:

# Large amount of credit goes to:
# https://github.com/keras-team/keras-contrib/blob/master/examples/improved_wgan.py
# which I've used as a reference for this implementation

from __future__ import print_function, division

from keras.datasets import mnist
from keras.layers.merge import _Merge
from keras.layers import Input, Dense, Reshape, Flatten, Dropout
from keras.layers import BatchNormalization, Activation, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
from keras.optimizers import RMSprop
from functools import partial

import keras.backend as K

import matplotlib.pyplot as plt

import sys

import numpy as np
import os
class RandomWeightedAverage(_Merge):
    """Provides a (random) weighted average between real and generated image samples"""
    def _merge_function(self, inputs):
        alpha = K.random_uniform((32, 1, 1, 1))
        return (alpha * inputs[0]) + ((1 - alpha) * inputs[1])

class WGANGP():
    def __init__(self):
        self.img_rows = 28
        self.img_cols = 28
        self.channels = 1
        self.img_shape = (self.img_rows, self.img_cols, self.channels)
        self.latent_dim = 100

        # Following parameter and optimizer set as recommended in paper
        self.n_critic = 5
        optimizer = RMSprop(lr=0.00005)

        # Build the generator and critic
        self.generator = self.build_generator()
        self.critic = self.build_critic()

        #-------------------------------
        # Construct Computational Graph
        #       for the Critic
        #-------------------------------

        # Freeze generator's layers while training critic
        self.generator.trainable = False

        # Image input (real sample)
        real_img = Input(shape=self.img_shape)

        # Noise input
        z_disc = Input(shape=(self.latent_dim,))
        # Generate image based of noise (fake sample)
        fake_img = self.generator(z_disc)

        # Discriminator determines validity of the real and fake images
        fake = self.critic(fake_img)
        valid = self.critic(real_img)

        # Construct weighted average between real and fake images
        interpolated_img = RandomWeightedAverage()([real_img, fake_img])
        # Determine validity of weighted sample
        validity_interpolated = self.critic(interpolated_img)

        # Use Python partial to provide loss function with additional
        # 'averaged_samples' argument
        partial_gp_loss = partial(self.gradient_penalty_loss,
                          averaged_samples=interpolated_img)
        partial_gp_loss.__name__ = 'gradient_penalty' # Keras requires function names

        self.critic_model = Model(inputs=[real_img, z_disc],
                            outputs=[valid, fake, validity_interpolated])
        self.critic_model.compile(loss=[self.wasserstein_loss,
                                              self.wasserstein_loss,
                                              partial_gp_loss],
                                        optimizer=optimizer,
                                        loss_weights=[1, 1, 10])
        #-------------------------------
        # Construct Computational Graph
        #         for Generator
        #-------------------------------

        # For the generator we freeze the critic's layers
        self.critic.trainable = False
        self.generator.trainable = True

        # Sampled noise for input to generator
        z_gen = Input(shape=(100,))
        # Generate images based of noise
        img = self.generator(z_gen)
        # Discriminator determines validity
        valid = self.critic(img)
        # Defines generator model
        self.generator_model = Model(z_gen, valid)
        self.generator_model.compile(loss=self.wasserstein_loss, optimizer=optimizer)


    def gradient_penalty_loss(self, y_true, y_pred, averaged_samples):
        """
        Computes gradient penalty based on prediction and weighted real / fake samples
        """
        gradients = K.gradients(y_pred, averaged_samples)[0]
        # compute the euclidean norm by squaring ...
        gradients_sqr = K.square(gradients)
        #   ... summing over the rows ...
        gradients_sqr_sum = K.sum(gradients_sqr,
                                  axis=np.arange(1, len(gradients_sqr.shape)))
        #   ... and sqrt
        gradient_l2_norm = K.sqrt(gradients_sqr_sum)
        # compute lambda * (1 - ||grad||)^2 still for each single sample
        gradient_penalty = K.square(1 - gradient_l2_norm)
        # return the mean as loss over all the batch samples
        return K.mean(gradient_penalty)


    def wasserstein_loss(self, y_true, y_pred):
        return K.mean(y_true * y_pred)

    def build_generator(self):

        model = Sequential()

        model.add(Dense(128 * 7 * 7, activation="relu", input_dim=self.latent_dim))
        model.add(Reshape((7, 7, 128)))
        model.add(UpSampling2D())
        model.add(Conv2D(128, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(UpSampling2D())
        model.add(Conv2D(64, kernel_size=4, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(Activation("relu"))
        model.add(Conv2D(self.channels, kernel_size=4, padding="same"))
        model.add(Activation("tanh"))

        model.summary()

        noise = Input(shape=(self.latent_dim,))
        img = model(noise)

        return Model(noise, img)

    def build_critic(self):

        model = Sequential()

        model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=self.img_shape, padding="same"))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
        model.add(ZeroPadding2D(padding=((0,1),(0,1))))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
        model.add(BatchNormalization(momentum=0.8))
        model.add(LeakyReLU(alpha=0.2))
        model.add(Dropout(0.25))
        model.add(Flatten())
        model.add(Dense(1))

        model.summary()

        img = Input(shape=self.img_shape)
        validity = model(img)

        return Model(img, validity)

    def train(self, epochs, batch_size, sample_interval=50):

        # Load the dataset
        (X_train, _), (_, _) = mnist.load_data()

        # Rescale -1 to 1
        X_train = (X_train.astype(np.float32) - 127.5) / 127.5
        X_train = np.expand_dims(X_train, axis=3)

        # Adversarial ground truths
        valid = -np.ones((batch_size, 1))
        fake =  np.ones((batch_size, 1))
        dummy = np.zeros((batch_size, 1)) # Dummy gt for gradient penalty
        nb_batches = int(X_train.shape[0] / batch_size)
        global_step = 0

        for epoch in range(epochs):

            for index in range(nb_batches):
                for _ in range(self.n_critic):
                    global_step += 1
                    imgs = X_train[index * batch_size:(index + 1) * batch_size]
                    # Sample generator input
                    noise = np.random.normal(0, 1, (batch_size, self.latent_dim))
                    # Train the critic
                    d_loss = self.critic_model.train_on_batch([imgs, noise],
                                                              [valid, fake, dummy])

                    # ---------------------
                    #  Train Generator
                    # ---------------------

                g_loss = self.generator_model.train_on_batch(noise, valid)

                # Plot the progress
                print("epoch%d step%d [D loss: %f] [G loss: %f]" % (epoch,global_step, d_loss[0], g_loss))

                # If at save interval => save generated image samples
                if global_step % sample_interval == 0:
                    self.sample_images(epoch,global_step)


    def sample_images(self, epoch,global_step):
        r, c = 10, 10
        noise = np.random.normal(0, 1, (r * c, self.latent_dim))
        gen_imgs = self.generator.predict(noise)

        # Rescale images 0 - 1
        gen_imgs = 0.5 * gen_imgs + 0.5

        fig, axs = plt.subplots(r, c)
        cnt = 0
        for i in range(r):
            for j in range(c):
                axs[i,j].imshow(gen_imgs[cnt, :,:,0], cmap='gray')
                axs[i,j].axis('off')
                cnt += 1
        if not os.path.isdir('images_wpgan'):
            os.mkdir('images_wpgan')
        fig.savefig("images_wpgan/epoch_%d_step_%d.png" % (epoch,global_step))
        plt.close()


if __name__ == '__main__':
    wgan = WGANGP()
    wgan.train(epochs=30, batch_size=32, sample_interval=200)


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_1 (Dense)              (None, 6272)              633472    
_________________________________________________________________
reshape_1 (Reshape)          (None, 7, 7, 128)         0         
_________________________________________________________________
up_sampling2d_1 (UpSampling2 (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
batch_normalization_1 (Batch (None, 14, 14, 128)       512       
_________________________________________________________________
activation_1 (Activation)    (None, 14, 14, 128)       0         
_________________________________________________________________
up_sampling2d_2 (UpSampling2 (None, 28, 28, 128)       0         
__________

/home/imi432_006/anaconda3/envs/tf/lib/python3.5/site-packages/keras/engine/training.py:490: UserWarning: Discrepancy between trainable weights and collected trainable weights, did you set `model.trainable` without calling `model.compile` after ?
  'Discrepancy between trainable weights and collected trainable'


epoch0 step5 [D loss: 32.180210] [G loss: -0.340254]
epoch0 step10 [D loss: 26.458223] [G loss: -0.185656]
epoch0 step15 [D loss: 19.545027] [G loss: -0.524759]
epoch0 step20 [D loss: 17.376942] [G loss: -0.439461]
epoch0 step25 [D loss: 13.673005] [G loss: -0.041736]
epoch0 step30 [D loss: 12.441192] [G loss: -0.129333]
epoch0 step35 [D loss: 10.851514] [G loss: -0.267667]
epoch0 step40 [D loss: 8.245709] [G loss: -0.136807]
epoch0 step45 [D loss: 9.094115] [G loss: -0.174859]
epoch0 step50 [D loss: 7.524340] [G loss: -0.406813]
epoch0 step55 [D loss: 4.259217] [G loss: -0.576437]
epoch0 step60 [D loss: 4.454852] [G loss: -0.481713]
epoch0 step65 [D loss: 8.179371] [G loss: -0.258607]
epoch0 step70 [D loss: 4.101719] [G loss: -0.341497]
epoch0 step75 [D loss: 2.452864] [G loss: -0.540236]
epoch0 step80 [D loss: 2.262691] [G loss: -0.580207]
epoch0 step85 [D loss: 2.468441] [G loss: -0.522113]
epoch0 step90 [D loss: 1.453644] [G loss: -0.560788]
epoch0 step95 [D loss: 1.976403] [G loss

epoch0 step755 [D loss: -0.948312] [G loss: -0.580182]
epoch0 step760 [D loss: -0.855478] [G loss: -0.935052]
epoch0 step765 [D loss: -1.328313] [G loss: -0.575787]
epoch0 step770 [D loss: -1.241110] [G loss: -0.780611]
epoch0 step775 [D loss: -0.898143] [G loss: -0.562402]
epoch0 step780 [D loss: -0.526099] [G loss: -0.744516]
epoch0 step785 [D loss: -0.792270] [G loss: -0.675472]
epoch0 step790 [D loss: -1.016057] [G loss: -1.233228]
epoch0 step795 [D loss: -0.567273] [G loss: -0.903295]
epoch0 step800 [D loss: -0.542904] [G loss: -0.832188]
epoch0 step805 [D loss: -0.753848] [G loss: -0.967990]
epoch0 step810 [D loss: -0.502298] [G loss: -0.905224]
epoch0 step815 [D loss: -0.720491] [G loss: -1.030318]
epoch0 step820 [D loss: -0.864567] [G loss: -0.724219]
epoch0 step825 [D loss: -1.008763] [G loss: -0.828813]
epoch0 step830 [D loss: -0.252902] [G loss: -0.796260]
epoch0 step835 [D loss: -0.827525] [G loss: -0.805426]
epoch0 step840 [D loss: -0.842914] [G loss: -0.606727]
epoch0 ste

...

epoch3 step30405 [D loss: -1.569278] [G loss: 5.548183]
epoch3 step30410 [D loss: -0.575111] [G loss: 4.507199]
epoch3 step30415 [D loss: -1.941437] [G loss: 5.570853]
epoch3 step30420 [D loss: -2.779068] [G loss: 5.080688]
epoch3 step30425 [D loss: -3.517038] [G loss: 5.381777]
epoch3 step30430 [D loss: -2.679531] [G loss: 5.708967]
epoch3 step30435 [D loss: -2.178589] [G loss: 5.888090]
epoch3 step30440 [D loss: -3.661220] [G loss: 5.717269]
epoch3 step30445 [D loss: -2.637399] [G loss: 5.569936]
epoch3 step30450 [D loss: -0.582716] [G loss: 5.664520]
epoch3 step30455 [D loss: -0.978069] [G loss: 6.529033]
epoch3 step30460 [D loss: -0.818428] [G loss: 5.731577]
epoch3 step30465 [D loss: -0.301012] [G loss: 6.394868]
epoch3 step30470 [D loss: -0.361257] [G loss: 4.991870]
epoch3 step30475 [D loss: -0.856662] [G loss: 5.454776]
epoch3 step30480 [D loss: -1.223637] [G loss: 5.823484]
epoch3 step30485 [D loss: -0.789886] [G loss: 5.611712]
epoch3 step30490 [D loss: -0.360138] [G loss: 5.

epoch3 step31145 [D loss: -0.832182] [G loss: 5.675760]
epoch3 step31150 [D loss: -0.825810] [G loss: 5.287624]
epoch3 step31155 [D loss: -0.188690] [G loss: 5.201423]
epoch3 step31160 [D loss: -1.194456] [G loss: 5.502564]
epoch3 step31165 [D loss: -0.948149] [G loss: 5.941034]
epoch3 step31170 [D loss: 0.276149] [G loss: 5.722682]
epoch3 step31175 [D loss: -0.512508] [G loss: 4.821754]
epoch3 step31180 [D loss: -0.449330] [G loss: 5.923141]
epoch3 step31185 [D loss: -0.499198] [G loss: 5.542386]
epoch3 step31190 [D loss: 0.416016] [G loss: 5.017941]
epoch3 step31195 [D loss: -0.816573] [G loss: 5.077569]
epoch3 step31200 [D loss: -1.011459] [G loss: 5.603032]
epoch3 step31205 [D loss: -0.574868] [G loss: 5.619380]
epoch3 step31210 [D loss: 0.414515] [G loss: 5.130091]
epoch3 step31215 [D loss: -1.509473] [G loss: 5.191543]
epoch3 step31220 [D loss: -0.737941] [G loss: 5.327587]
epoch3 step31225 [D loss: -0.425295] [G loss: 6.042472]
epoch3 step31230 [D loss: -1.213654] [G loss: 5.735

epoch3 step31885 [D loss: -0.326727] [G loss: 3.436830]
epoch3 step31890 [D loss: -0.184805] [G loss: 3.008448]
epoch3 step31895 [D loss: -1.203214] [G loss: 3.764196]
epoch3 step31900 [D loss: 0.131993] [G loss: 2.553176]
epoch3 step31905 [D loss: -0.549102] [G loss: 2.804532]
epoch3 step31910 [D loss: -0.479421] [G loss: 3.042911]
epoch3 step31915 [D loss: -0.554652] [G loss: 2.743864]
epoch3 step31920 [D loss: 0.373612] [G loss: 3.015362]
epoch3 step31925 [D loss: -0.411447] [G loss: 3.167982]
epoch3 step31930 [D loss: -1.328186] [G loss: 2.639614]
epoch3 step31935 [D loss: -0.642894] [G loss: 2.351488]
epoch3 step31940 [D loss: -0.206078] [G loss: 3.049723]
epoch3 step31945 [D loss: -0.903631] [G loss: 2.922047]
epoch3 step31950 [D loss: -0.391625] [G loss: 2.934031]
epoch3 step31955 [D loss: 0.212141] [G loss: 3.339913]
epoch3 step31960 [D loss: -0.288465] [G loss: 3.804418]
epoch3 step31965 [D loss: -1.115402] [G loss: 3.649343]
epoch3 step31970 [D loss: -0.356282] [G loss: 3.280

epoch3 step32625 [D loss: -1.882733] [G loss: 4.746539]
epoch3 step32630 [D loss: -2.030539] [G loss: 5.751969]
epoch3 step32635 [D loss: -2.707870] [G loss: 5.215613]
epoch3 step32640 [D loss: -0.807741] [G loss: 5.033519]
epoch3 step32645 [D loss: -0.799804] [G loss: 5.426315]
epoch3 step32650 [D loss: -1.161317] [G loss: 5.339447]
epoch3 step32655 [D loss: -1.073292] [G loss: 5.370911]
epoch3 step32660 [D loss: -1.721346] [G loss: 5.305784]
epoch3 step32665 [D loss: -1.076966] [G loss: 5.595442]
epoch3 step32670 [D loss: -1.468065] [G loss: 5.556506]
epoch3 step32675 [D loss: -0.824740] [G loss: 4.809675]
epoch3 step32680 [D loss: -0.429903] [G loss: 5.293478]
epoch3 step32685 [D loss: -1.097258] [G loss: 4.748616]
epoch3 step32690 [D loss: -1.690568] [G loss: 4.472808]
epoch3 step32695 [D loss: -1.977430] [G loss: 4.124202]
epoch3 step32700 [D loss: -0.383516] [G loss: 4.906452]
epoch3 step32705 [D loss: -0.973012] [G loss: 5.229792]
epoch3 step32710 [D loss: -0.104494] [G loss: 5.

KeyboardInterrupt: 